# SERAB demo
* This notebook decomposes `clf_benchmark.py`, which evaluate a given pre-trained model against SERAB to show the main steps behind the script.



In [ ]:
!git clone https://github.com/Neclow/SERAB.git

fatal: destination path 'SERAB' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/Neclow/SERAB.git
!cd SERAB/SERAB/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/config.yaml && patch --ignore-whitespace < config.diff
!cd SERAB/SERAB/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/train.py && patch < train.diff
!cd SERAB/SERAB/byol_a/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/augmentations.py && patch < augmentations.diff
!cd SERAB/SERAB/byol_a/ &&curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/common.py && patch < common.diff
!cd SERAB/SERAB/byol_a/ &&curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/dataset.py && patch < dataset.diff
!cd SERAB/SERAB/byol_a/ && curl -O https://raw.githubusercontent.com/nttcslab/byol-a/f2451c366d02be031a31967f494afdf3485a85ff/byol_a/models.py && mv models.py models/audio_ntt.py

fatal: destination path 'SERAB' already exists and is not an empty directory.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   564  100   564    0     0   1734      0 --:--:-- --:--:-- --:--:--  1730
patching file config.yaml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4867  100  4867    0     0  20651      0 --:--:-- --:--:-- --:--:-- 20710
patching file train.py
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8273  100  8273    0     0  30096      0 --:--:-- --:--:-- --:--:-- 30193
patching file augmentations.py
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   To

## Install libraries

In [ ]:
!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
!pip install -q opensmile

!pip install -q pytorch-lightning==1.4.0 torchmetrics==0.6.0
!pip install -q torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext==0.12.0
!pip install -q einops==0.4.0
!pip install -q pydub==0.25.1
!pip install -q tensorflow_datasets==4.3.0
!pip install -q librosa==0.8.1
!pip install --upgrade librosa
!pip install torch==1.12.1 torchtext==0.13.1 pytorch-lightning==1.9.3
!pip install torchaudio==0.12.1

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.1
    Uninstalling librosa-0.8.1:
      Successfully uninstalled librosa-0.8.1
  Using cached torch-1.12.1-cp310-cp310-manylinux1_x86_64.whl.metadata (22 kB)
  Using cached torchtext-0.13.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.9 kB)
  Using cached pytorch_lightning-1.9.3-py3-none-any.whl.metadata (22 kB)
  Using cached torchmetrics-1.5.1-py3-none-any.whl.metadata (20 kB)
Using cached torch-1.12.1-cp310-cp310-manylinux1_x86_64.whl (776.3 MB)
Using cached torchtext-0.13.1-cp310-cp310-manylinux1_x86_64.whl (1.9 MB)
Using cached pytorch_lightning-1.9.3-py3-none-any.whl (826 kB)
Using cached torchmetrics-1.5.1-py3-none-any.whl (890 kB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-

## Load imports and config variables

In [ ]:
SERAB_PATH = 'SERAB/SERAB/'

import sys

sys.path.append(SERAB_PATH)

In [ ]:
import collections
import os

import librosa
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import torch





In [ ]:
from pytorch_lightning.utilities.seed import seed_everything
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import recall_score
from torchaudio.transforms import MelSpectrogram

In [ ]:
from byol_a.common import load_yaml_config
from byol_a.augmentations import PrecomputedNorm
from clf_benchmark import dat_from_split, get_sklearn_models
from settings import CLF_STATS_DICT, RANDOM_SEED, REQUIRED_SAMPLE_RATE
from utils import compute_norm_stats, generate_embeddings, load_model, save_results, speaker_normalization

In [ ]:
seed_everything(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = load_yaml_config(f'{SERAB_PATH}/config.yaml')
to_melspec = MelSpectrogram(
    sample_rate=cfg.sample_rate,
    n_fft=cfg.n_fft,
    win_length=cfg.win_length,
    hop_length=cfg.hop_length,
    n_mels=cfg.n_mels,
    f_min=cfg.f_min,
    f_max=cfg.f_max,
)
results = {}

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/seed.py:47: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_fabric.utilities.seed:Global seed set to 42


## Load data
* If you want to test a model on a dataset other than CREMA-D you will need to download and build the dataset.
    * An example of how to proceed with emoDB is shown below. Uncomment the lines below (`UNCOMMENT CODE HERE`) to build the emodB dataset.
        * You will need to create the subfolders `downloads/manual` under `tensorflow_datasets` and paste the compressed emodB dataset in there.
* For this demo, we will use CREMA-D as already pre-built in TFDS.

In [ ]:
# Create tensorflow_datasets folders
!mkdir -p ../root/tensorflow_datasets/
!mkdir -p ../root/tensorflow_datasets/downloads/
!mkdir -p ../root/tensorflow_datasets/downloads/manual

## UNCOMMENT CODE HERE
## Copy compressed dataset files into download/manual
# !cp -n drive/MyDrive/SERAB/tensorflow_datasets/downloads/manual/*.zip ../root/tensorflow_datasets/downloads/manual/

## Copy the emoDB TFDS scripts and build the emoDB dataset
# !cp -r -n SERAB/tensorflow_datasets/emoDB ../root/tensorflow_datasets/emoDB
# !cd ../root/tensorflow_datasets/emoDB && tfds build emoDB

In [ ]:
dataset_name = 'crema_d'

In [ ]:
model_selection = 'predefined'

SingleSplit = collections.namedtuple('SingleSplit', ['audio', 'labels', 'speaker_id'])
Data = collections.namedtuple('Data', ['train', 'validation', 'test'])
all_data = Data(
    train=SingleSplit(*dat_from_split(dataset_name, 'train')),
    validation=SingleSplit(*dat_from_split(dataset_name, 'validation')),
    test=SingleSplit(*dat_from_split(dataset_name, 'test'))
)

orig_sr = tfds.builder(dataset_name).info.features['audio'].sample_rate
num_classes = len(np.unique(all_data.train.labels))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

...

Generating validation examples...:   0%|          | 0/738 [00:00<?, ? examples/s]

Shuffling crema_d-validation.tfrecord...:   0%|          | 0/738 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/1556 [00:00<?, ? examples/s]

Shuffling crema_d-test.tfrecord...:   0%|          | 0/1556 [00:00<?, ? examples/s]

Dataset crema_d downloaded and prepared to /root/tensorflow_datasets/crema_d/1.0.0. Subsequent calls will reuse this data.
Finished train
Finished validation
Finished test


In [ ]:
# Load data statistics
try:
    stats = CLF_STATS_DICT[dataset_name]
except KeyError:
    print(f'Did not find mean/std stats for {dataset_name}.')
    stats = compute_norm_stats(dataset_name, all_data.train.audio, orig_sr, to_melspec)

    CLF_STATS_DICT[dataset_name] = stats

    print(CLF_STATS_DICT)
normalizer = PrecomputedNorm(stats)

## Load the model
* For this demo, we will use "BYOL-S", a re-trained BYOL-A on speech samples of AudioSet with 512 features.
    * More information regarding the models can be found in the pre-print (https://arxiv.org/abs/2110.03414)

In [ ]:
model_name = 'default'
ckpt_folder = f'{SERAB_PATH}/checkpoints/'

model, weight_file = load_model(model_name, cfg, device, ckpt_folder)

print(weight_file)

SERAB/SERAB/checkpoints/default1024_BYOLAs64x96-2107292000-e100-bs256-lr0003-rs42.pth


In [ ]:
# Generate embeddings
embeddings = Data(
    train=generate_embeddings(
        model,
        model_name,
        all_data.train.audio,
        'train',
        orig_sr,
        to_melspec,
        normalizer,
        device
    ),
    validation=generate_embeddings(
        model,
        model_name,
        all_data.validation.audio,
        'validation',
        orig_sr,
        to_melspec,
        normalizer,
        device
    ),
    test=generate_embeddings(
        model,
        model_name,
        all_data.test.audio,
        'test',
        orig_sr,
        to_melspec,
        normalizer,
        device
    )
)

Generating embeddings for train: 100%|##########| 5144/5144 [02:28<00:00, 34.66it/s]


Finished train


Generating embeddings for validation: 100%|##########| 738/738 [00:20<00:00, 35.96it/s]


Finished validation


Generating embeddings for test: 100%|##########| 1556/1556 [00:43<00:00, 36.02it/s]

Finished test


In [ ]:
print(embeddings.train.mean(), embeddings.train.std())
print(embeddings.validation.mean(), embeddings.validation.std())
print(embeddings.test.mean(), embeddings.test.std())
print(embeddings.train.shape)
print(embeddings.validation.shape)
print(embeddings.test.shape)

# Load classifiers
log_list, estimator_list, param_list = get_sklearn_models()

4.344951 4.3818913
4.3764834 4.4149346
4.289814 4.375481
(5144, 1024)
(738, 1024)
(1556, 1024)


In [ ]:
# Speaker normalization
# Can also try with normal standardization (StandardScaler), should not change the results too much
normalized_train = speaker_normalization(embeddings.train, all_data.train.speaker_id)
normalized_validation = speaker_normalization(embeddings.validation, all_data.validation.speaker_id)
normalized_test = speaker_normalization(embeddings.test, all_data.test.speaker_id)

# Aggregate labels and speaker IDs
normalized_train = np.append(normalized_train, normalized_validation, axis=0)
labels_train = np.append(all_data.train.labels, all_data.validation.labels, axis=0)
speaker_id_train = np.append(all_data.train.speaker_id, all_data.validation.speaker_id, axis=0)

In [ ]:
# Initialize an empty dictionary to store results
results = {}

for i, (estimator_name, estimator, param_grid) in enumerate(zip(log_list, estimator_list, param_list)):
    print(f'Step {i+1}/{len(estimator_list)}: {estimator_name}...')

    # Use GridSearchCV only if 'predefined' is selected
    if model_selection == 'predefined':
        split_indices = np.concatenate([
            np.full(embeddings.train.shape[0], -1),
            np.zeros(embeddings.validation.shape[0])
        ])
        split = PredefinedSplit(split_indices)
        clf = GridSearchCV(
            estimator=estimator,
            param_grid=param_grid,
            cv=split,
            n_jobs=-1,
            refit=True,
            verbose=0,
            error_score='raise'  # Ensures all errors are raised for easier debugging
        )
    else:
        clf = estimator  # Use the estimator directly without parameter tuning

    # Fit the model and evaluate accuracy and recall
    try:
        clf.fit(normalized_train, labels_train)
        test_acc = clf.score(normalized_test, all_data.test.labels)
        test_uar = recall_score(all_data.test.labels, clf.predict(normalized_test), average='macro')

        # Store results in dictionary format
        results[estimator_name] = {
            'test_acc': test_acc,
            'test_uar': test_uar
        }
        print('Done')

    except ValueError as e:
        print(f"Error with {estimator_name}: {e}")
        continue

# Create a DataFrame from the results, rounding to 1 decimal place
results_df = pd.DataFrame(results).applymap(lambda x: round(x * 100, 1))

# Select the model with the highest test accuracy
best_estimator = results_df.loc['test_acc'].idxmax()
results_df = results_df[[best_estimator]].rename(columns={best_estimator: dataset_name})

# Display the final results DataFrame
print(results_df)


Step 1/5: LDA...
Done
Step 2/5: LR...
Error with LR: The 'max_iter' parameter of LogisticRegression must be an int in the range [0, inf). Got 10000.0 instead.
Step 3/5: QDA...


/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:947: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Done
Step 4/5: RF...
Done
Step 5/5: SVC...
Error with SVC: The 'max_iter' parameter of SVC must be an int in the range [-1, inf). Got 10000.0 instead.
          crema_d
test_acc     71.3
test_uar     71.5


<ipython-input-19-ef266557d439>:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_df = pd.DataFrame(results).applymap(lambda x: round(x * 100, 1))


In [ ]:
results_df

,crema_d
test_acc,71.3
test_uar,71.5


In [ ]:
filename = os.path.splitext(os.path.basename(weight_file))[0] if weight_file else model_name
results_folder = f'{SERAB_PATH}/clf_results/'
save_results(filename + '.csv', results_df, results_folder)

File default1024_BYOLAs64x96-2107292000-e100-bs256-lr0003-rs42.csv does not exist yet. Creating a new results file.


In [ ]:
pd.read_csv(f'{SERAB_PATH}/clf_results/default1024_BYOLAs64x96-2107292000-e100-bs256-lr0003-rs42.csv', index_col=0)

,crema_d
test_acc,71.3
test_uar,71.5
